Here I will try out different cloud solutions to query and see how they perform

In [11]:
import os
import json
from dotenv import load_dotenv

import weaviate
from weaviate.auth import Auth
from weaviate.classes.config import Configure, Property, DataType

load_dotenv()

# Configuration
JSONL_PATH = "data/t2d_guideline_ee/30_chunks/2-tuubi-diabeedi-diagnostika-ravi.jsonl"
COLLECTION_NAME = "Guideline"

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")

if not WEAVIATE_URL:
    raise ValueError("Missing WEAVIATE_URL in environment.")
if not WEAVIATE_API_KEY:
    raise ValueError("Missing WEAVIATE_API_KEY in environment.")


# Code below adds the JSONL objects to Weaviate collection.
def load_jsonl_objects(path: str) -> list[dict]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")

    objects: list[dict] = []
    with open(path, "r", encoding="utf-8") as f:
        for line_no, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            try:
                data = json.loads(line)
            except json.JSONDecodeError as e:
                raise ValueError(f"Invalid JSON on line {line_no}: {e}") from e

            # Support both:
            # 1) {"properties": {...}}  and 2) {...flat props...}
            props = data.get("properties", data)
            if not isinstance(props, dict):
                raise ValueError(f"Expected dict properties on line {line_no}, got {type(props)}")

            objects.append(props)

    return objects


with weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
) as client:

    # --- PART 1: CREATE COLLECTION ONLY IF MISSING ---
    if not client.collections.exists(COLLECTION_NAME):
        client.collections.create(
            name=COLLECTION_NAME,
            vector_config=Configure.Vectors.text2vec_weaviate(
                name="default",
                source_properties=["search_text"],
                vectorize_collection_name=False,
            ),
            properties=[
                Property(name="chunk_id", data_type=DataType.INT),
                Property(name="search_text", data_type=DataType.TEXT),
                Property(name="text", data_type=DataType.TEXT),
                Property(name="breadcrumbs", data_type=DataType.TEXT),
                Property(name="source", data_type=DataType.TEXT),
                Property(name="year", data_type=DataType.INT),
                Property(name="language", data_type=DataType.TEXT),
                Property(name="version_id", data_type=DataType.TEXT),
            ],
        )
        print(f"✅ Created collection: '{COLLECTION_NAME}'")
    else:
        print(f"ℹ️ Collection already exists: '{COLLECTION_NAME}' (will append new objects)")

    # --- PART 2: IMPORT OBJECTS (APPEND) ---
    guideline_collection = client.collections.use(COLLECTION_NAME)

    objects_to_add = load_jsonl_objects(JSONL_PATH)
    print(f"📦 Loaded {len(objects_to_add)} objects from JSONL")

    # Batch import (append)
    with guideline_collection.batch.fixed_size(batch_size=200) as batch:
        for props in objects_to_add:
            batch.add_object(properties=props)

    # --- PART 3: CHECK IMPORT STATUS ---
    failed = guideline_collection.batch.failed_objects
    if failed:
        print(f"❌ Failed to import {len(failed)} objects.")
        print("First failure:")
        print(failed[0])
    else:
        total = guideline_collection.aggregate.over_all(total_count=True).total_count
        print(f"🚀 Import complete. Collection '{COLLECTION_NAME}' now has {total} objects total.")


🗑️ Deleted existing collection: 'Guideline'
✅ Created clean collection: 'Guideline'
📦 Loaded 26 objects from JSONL
🚀 Successfully imported 26 objects into 'Guideline'


In [19]:
# Lets test different query setups
# Start with help function
def show_results(title, resp, fields=("chunk_id","search_text","source","year","language"), max_text_chars=260):
    print("\n" + "="*90)
    print(title)
    print("="*90)

    if not resp.objects:
        print("No results.")
        return

    for i, obj in enumerate(resp.objects, start=1):
        props = obj.properties or {}
        print(f"\n[{i}] uuid={obj.uuid}")
        for f in fields:
            print(f"  {f}: {props.get(f)}")
        text = props.get("search_text") or ""
        text = text.replace("\n", " ").strip()
        if len(text) > max_text_chars:
            text = text[:max_text_chars] + "…"
        print(f"  search_text (snippet): {text}")

with weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
) as client:
    col = client.collections.use("Guideline")

    # Put your test query here:
    q = "2. tüübi diabeedi ravi metformiin"

    # You can run any of the strategies below inside this context.
    resp = col.query.near_text(
    query=q,
    limit=5,
    return_properties=["chunk_id","search_text","source","year","language"]
)
show_results("A) near_text (semantic)", resp)



A) near_text (semantic)

[1] uuid=533a0d75-6996-4542-ae3b-a7c9243418a6
  chunk_id: 7
  search_text: Ravijuhendi soovituste loetelu > Farmakoteraapia alustamine 2. tüüpi diabeedi diagnoosiga patsiendil, kel puudub SVH lisarisk
7. **2. tüüpi diabeedi diagnoosiga patsiendil, kes vajab farmakoteraapiat ja kellel ei esine püsiva proteinuuriaga² kroonilist neeruhaigust või dokumenteeritud südame vasaku vatsakese langenud väljutusfraktsiooniga³ südamepuudulikkust, alustage ravi metformiiniga.**
Tugev positiivne soovitus, väga madal tõendatuse aste
8. **2. tüüpi diabeedi diagnoosiga patsiendil alustage metformiinravi annuses vähemalt 500 mg kaks korda päevas. Vajaduse korral suurendage annust mitme nädala jooksul järk-järgult, et vähendada gastrointestinaalsete kõrvaltoimete riski.**
Praktiline soovitus
9. **2. tüüpi diabeedi diagnoosiga patsiendil, kellel on metformiin vastunäidustatud või kes seda ei talu, kaaluge farmakoteraapia alustamist sulfonüüluurea preparaadi, DPP4 inhibiitori, SGLT2

In [87]:
import os
import pandas as pd
from dotenv import load_dotenv

import weaviate
from weaviate.auth import Auth

from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, ConfigDict

load_dotenv()

assert os.getenv("WEAVIATE_URL")
assert os.getenv("WEAVIATE_API_KEY")
assert os.getenv("OPENAI_API_KEY")

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
COLLECTION_NAME = "Guideline"


In [88]:
def retrieve_all_strategies(
    query: str,
    k: int = 5,
    hybrid_alphas=(0.2, 0.5, 0.8),
):
    rows = []

    with weaviate.connect_to_weaviate_cloud(
        cluster_url=WEAVIATE_URL,
        auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    ) as client:
        col = client.collections.use(COLLECTION_NAME)

        props = ["search_text", "breadcrumbs", "chunk_id", "source", "year"]

        # BM25
        resp = col.query.bm25(query=query, limit=k, return_properties=props)
        rows.append(("bm25", None, resp.objects))

        # near_text
        resp = col.query.near_text(query=query, limit=k, return_properties=props)
        rows.append(("near_text", None, resp.objects))

        # hybrid sweep
        for a in hybrid_alphas:
            resp = col.query.hybrid(query=query, alpha=a, limit=k, return_properties=props)
            rows.append(("hybrid", a, resp.objects))

    return rows


In [89]:
class RetrievalJudgeResult(BaseModel):
    model_config = ConfigDict(extra="forbid")

    retrieval_relevance: int = Field(ge=1, le=5)
    coverage: int = Field(ge=1, le=5)
    groundedness: int = Field(ge=1, le=5)
    noise: int = Field(ge=1, le=5, description="5 = very noisy, 1 = very clean")
    overall: int = Field(ge=1, le=5)

    verdict: str
    issues: list[str]
    notes: str


judge_llm = ChatOpenAI(
    model="gpt-5.2",
    temperature=0,
)

judge_structured = judge_llm.with_structured_output(
    RetrievalJudgeResult,
    method="json_schema",
    strict=True,
)

JUDGE_PROMPT = """
Developer: Evaluate SEARCH RESULTS for a given QUERY, focusing on the relevance and quality of retrieved content—not answering the query itself.

Inputs:
- QUERY: Search query string.
- RETRIEVED CHUNKS: List of retrieved content strings.

Checklist:
- Review QUERY and all RETRIEVED CHUNKS.
- Assess relevance of each chunk to the query.
- Determine if chunks collectively address the query's scope.
- Check for substantive (not superficial) topic coverage.
- Identify any irrelevant or noisy content.
- Assign scores per evaluation dimension.
- Synthesize results and assign a verdict.

Scoring (1–5 integers):
- retrieval_relevance: Do chunks address the query?
- coverage: Do chunks cover the full query intent?
- groundedness: Are contents deeply connected to the true topic?
- noise: Amount of unrelated/misleading content (5 = very noisy, 1 = almost none).
- overall: Holistic retrieval quality.

Verdict rules:
- good: overall ≥ 4 AND retrieval_relevance ≥ 4 AND noise ≤ 2
- bad: overall ≤ 2 OR retrieval_relevance ≤ 2
- borderline: otherwise

Before finalizing, check that scores and verdict match these rules; adjust if needed.

Output ONLY this JSON structure, in order:
- retrieval_relevance
- coverage
- groundedness
- noise
- overall
- verdict

Example:
{
  "retrieval_relevance": 4,
  "coverage": 5,
  "groundedness": 4,
  "noise": 1,
  "overall": 4,
  "verdict": "good"
}

If QUERY or RETRIEVED CHUNKS is missing or invalid, return ONLY:
{
  "error": "Description of the problem."
}

Strictly follow the schema; no extra fields or output.
"""

def judge_retrieval(query: str, chunks: list[str]) -> dict:
    context = "\n\n".join(f"[{i+1}] {c}" for i, c in enumerate(chunks))

    msg = f"""
QUERY:
{query}

RETRIEVED CHUNKS:
{context}
"""
    res = judge_structured.invoke(
        [
            {"role": "system", "content": JUDGE_PROMPT},
            {"role": "user", "content": msg},
        ]
    )
    return res.model_dump()


/Users/nikitaumov/Documents/code/weekend_projects/learning_splitting/venv/lib/python3.14/site-packages/langchain_core/runnables/utils.py:756: DeprecationWarning: 'asyncio.iscoroutinefunction' is deprecated and slated for removal in Python 3.16; use inspect.iscoroutinefunction() instead
  return asyncio.iscoroutinefunction(func) or (
/Users/nikitaumov/Documents/code/weekend_projects/learning_splitting/venv/lib/python3.14/site-packages/langchain_core/runnables/utils.py:758: DeprecationWarning: 'asyncio.iscoroutinefunction' is deprecated and slated for removal in Python 3.16; use inspect.iscoroutinefunction() instead
  and asyncio.iscoroutinefunction(func.__call__)


In [95]:
queries = [
    "Millal tuleb 2. tüüpi diabeediga patsient suunata endokrinoloogi vastuvõtule?",
]


In [96]:
rows = []

for q in queries:
    all_results = retrieve_all_strategies(q, k=5)

    for strategy, alpha, objs in all_results:
        chunks = []
        for obj in objs:
            p = obj.properties or {}
            chunks.append(p.get("search_text") or "")

        judgment = judge_retrieval(q, chunks)

        for rank, obj in enumerate(objs, start=1):
            p = obj.properties or {}
            rows.append({
                "query": q,
                "strategy": strategy,
                "alpha": alpha,
                "rank": rank,
                "chunk_id": p.get("chunk_id"),
                "breadcrumbs": p.get("breadcrumbs"),
                "search_text": p.get("search_text"),
                **judgment,
            })


In [97]:
df = pd.DataFrame(rows)

df = df.sort_values(
    ["query", "strategy", "alpha", "rank"]
)

df[
    [
        "query",
        "strategy",
        "alpha",
        "rank",
        "chunk_id",
        "breadcrumbs",
        "search_text",
    ]
]
df["alpha_label"] = df.apply(
    lambda r: (
        f"{r['alpha']}"
        if r["strategy"] == "hybrid"
        else "N/A"
    ),
    axis=1,
)

In [98]:
from ipywidgets import Button, VBox, HBox, Output
from IPython.display import display, HTML
import pandas as pd
import html

# ===============================
# SAFETY & PREP
# ===============================
required_cols = {
    "query", "strategy", "alpha_label", "rank",
    "chunk_id", "breadcrumbs", "search_text"
}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"df is missing required columns: {missing}")

# Ensure relevance column exists
if "human_relevance" not in df.columns:
    df["human_relevance"] = pd.NA

# Normalize relevance column
df["human_relevance"] = pd.to_numeric(
    df["human_relevance"], errors="coerce"
).fillna(-1).astype(int)

# Reset index for navigation
df = df.reset_index(drop=True)

# ===============================
# STATE
# ===============================
current_idx = 0
out = Output()

# ===============================
# UI ELEMENTS
# ===============================
prev_btn = Button(description="⬅ Previous")
next_btn = Button(description="Next ➡")
save_csv_btn = Button(description="💾 Save to CSV", button_style="warning")

btn_0 = Button(description="0", button_style="danger", tooltip="Irrelevant")
btn_1 = Button(description="1", tooltip="Weakly relevant")
btn_2 = Button(description="2", button_style="info", tooltip="Relevant")
btn_3 = Button(description="3", button_style="success", tooltip="Highly relevant")

rating_buttons = HBox([btn_0, btn_1, btn_2, btn_3])

# ===============================
# RENDER FUNCTION (HTML, NOT MARKDOWN)
# ===============================
def render_row(idx):
    row = df.loc[idx]

    # Escape text for HTML safety
    query = html.escape(str(row["query"]))
    search_text = html.escape(str(row["search_text"]))
    breadcrumbs = html.escape(str(row["breadcrumbs"]))
    rating = row["human_relevance"]

    with out:
        out.clear_output()
        display(HTML(f"""
        <div style="font-family: monospace; line-height: 1.5;">
            <h2>🔎 Retrieval Review ({idx+1} / {len(df)})</h2>

            <b>Query:</b>
            <div style="margin-bottom: 10px;">{query}</div>

            <b>Strategy:</b> {row['strategy']}<br>
            <b>Alpha:</b> {row['alpha_label']}<br>
            <b>Rank:</b> {row['rank']}<br>
            <b>Chunk ID:</b> {row['chunk_id']}<br>
            <b>Breadcrumbs:</b> {breadcrumbs}

            <hr>

            <h3>search_text</h3>
            <pre style="
                white-space: pre-wrap;
                background: #f8f8f8;
                border: 1px solid #ddd;
                padding: 12px;
                max-height: 400px;
                overflow-y: auto;
            ">{search_text}</pre>

            <hr>

            <b>Current rating:</b> {rating if rating >= 0 else "not rated"}<br>
            <small>0 = irrelevant · 1 = weak · 2 = relevant · 3 = highly relevant</small>
        </div>
        """))

# ===============================
# HANDLERS
# ===============================
def set_rating(value):
    global current_idx
    df.loc[current_idx, "human_relevance"] = value
    render_row(current_idx)

def prev_row(_):
    global current_idx
    if current_idx > 0:
        current_idx -= 1
        render_row(current_idx)

def next_row(_):
    global current_idx
    if current_idx < len(df) - 1:
        current_idx += 1
        render_row(current_idx)

def save_csv(_):
    df.to_csv("manual_retrieval_review.csv", index=False)
    with out:
        display(HTML("<b>💾 Saved to manual_retrieval_review.csv</b>"))

# Button bindings
btn_0.on_click(lambda _: set_rating(0))
btn_1.on_click(lambda _: set_rating(1))
btn_2.on_click(lambda _: set_rating(2))
btn_3.on_click(lambda _: set_rating(3))

prev_btn.on_click(prev_row)
next_btn.on_click(next_row)
save_csv_btn.on_click(save_csv)

# ===============================
# DISPLAY
# ===============================
controls = HBox([prev_btn, next_btn, rating_buttons, save_csv_btn])
display(VBox([controls, out]))

render_row(current_idx)


In [99]:


summary = (
    df.groupby(["strategy", "alpha_label"])
    .agg(
        overall_mean=("overall", "mean"),
        relevance_mean=("retrieval_relevance", "mean"),
        coverage_mean=("coverage", "mean"),
        groundedness_mean=("groundedness", "mean"),
        noise_mean=("noise", "mean"),
        good_rate=("verdict", lambda s: (s == "good").mean()),
        n=("overall", "count"),
    )
    .reset_index()
    .sort_values("overall_mean", ascending=False)
)

summary


,strategy,alpha_label,overall_mean,relevance_mean,coverage_mean,groundedness_mean,noise_mean,good_rate,n
2,hybrid,0.5,5.0,5.0,4.0,5.0,1.0,1.0,5
3,hybrid,0.8,5.0,5.0,4.0,5.0,1.0,1.0,5
4,near_text,N/A,5.0,5.0,5.0,5.0,1.0,1.0,5
0,bm25,N/A,4.0,5.0,4.0,5.0,2.0,1.0,5
1,hybrid,0.2,4.0,5.0,4.0,5.0,2.0,1.0,5
